<a href="https://colab.research.google.com/github/GU-DPM/EvolutionaryClassifier/blob/main/analysis/EC_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

if running on colab...

In [1]:
from google.colab import auth
from google.colab import data_table
data_table.enable_dataframe_formatter()
auth.authenticate_user()

otherwise just start here if running from VM...

In [2]:
from google.cloud import bigquery
project = 'mccoylab' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)

In [3]:
import pandas as pd
import seaborn as sns

Define benefit query

In [18]:
# build benefit query
# firstSame, secondSame, bothDiff, bothSame
benefit_def = 'secondSame'
num_records = 50

#benefit_query = """
#SELECT *
#  FROM `mccoylab.dpm_dev.dpmTrial2019_benefitSummary` dpm
# WHERE dpm.Category = \'""" + benefit_def  + """\'
#ORDER BY RAND() LIMIT """ + str(num_records) + """
#"""

#collect sample population from whole dataset
#benefit_query = """
#SELECT *
#  FROM `mccoylab.dpm_dev.dpmTrial2019_benefitSummary` dpm
#ORDER BY RAND() LIMIT """ + str(num_records) + """
#"""

#collect sample population from patients that benefit on trial
#benefit_query = """
#SELECT *
#  FROM `mccoylab.dpm_dev.dpmTrial2019_benefitSummary` dpm
# WHERE dpm.improvementDPMtrial is true
#ORDER BY RAND() LIMIT """ + str(num_records) + """
#"""





benefit_query

'\nSELECT *\n  FROM `mccoylab.dpm_dev.dpmTrial2019_benefitSummary` dpm\n WHERE dpm.improvementDPMtrial is true\nORDER BY RAND() LIMIT 50\n'

In [19]:
# sample results for virtual patients that neet benefit definiton

benefit_df = client.query(benefit_query).to_dataframe()
benefit_df

,int64_field_0,paramID,Category,Surv_0,Surv_2_2,Surv_2_2trial,Drug1Step1_0,Drug2Step1_0,Drug1Step2_0,Drug2Step2_0,Drug1Step1_2_2,Drug2Step1_2_2,Drug1Step2_2_2,Drug2Step2_2_2,improvementDays_DPM,improvementPercent_DPM,improvementDays_DPMtrial,improvementPercent_DPMtrial,improvementDPM,improvementDPMtrial
0,1570361,23052505,bothDiff,90,1845,1845,0,1,1,0,0.5,0.5,0.0,1.0,1755,19.500000,1755,19.500000,True,True
1,2173523,30213422,bothDiff,945,1845,1845,0,1,0,1,1.0,0.0,1.0,0.0,900,0.952381,900,0.952381,True,True
2,770110,12936691,secondSame,675,1845,1845,0,1,0,1,1.0,0.0,0.0,1.0,1170,1.733333,1170,1.733333,True,True
3,2370651,32107105,bothDiff,0,90,90,0,1,-1,-1,1.0,0.0,0.5,0.5,90,inf,90,inf,True,True
4,2893524,37763999,bothDiff,180,1845,1845,1,0,1,0,0.5,0.5,0.0,1.0,1665,9.250000,1665,9.250000,True,True
5,1797346,26096527,secondSame,675,1845,1845,0,1,0,1,1.0,0.0,0.0,1.0,1170,1.733333,1170,1.733333,True,True
6,36847,821539,bothDiff,45,1845,1845,1,0,1,0,0.0,1.0,0.0,1.0,1800,40.000000,1800,40.000000,True,True
7,2885572,37746135,bothDiff,270,1845,1845,1,0,1,0,0.5,0.5,0.0,1.0,1575,5.833333,1575,5.833333,True,True
8,2508733,33642917,bothDiff,270,1845,1845,0,1,1,0,0.5,0.5,0.0,1.0,1575,5.833333,1575,5.833333,True,True
9,967717,15612208,bothDiff,0,1845,1845,1,0,-1,-1,0.5,0.5,0.0,1.0,1845,inf,1845,inf,True,True


In [9]:
# build state variable vector
paramID_list = ",".join(str(x) for x in benefit_df.paramID.tolist())

stateVar_query = """
SELECT pop.parameter_index, pop.strategy_name,
       pop.s_t_0, pop.r1_t_0, pop.r2_t_0, pop.r12_t_0,
       pop.s_t_45, pop.r1_t_45, pop.r2_t_45, pop.r12_t_45,
 FROM `mccoylab.dpm_dev.pop-results-2019` pop
 WHERE pop.parameter_index IN (""" + paramID_list + """)
"""
stateVar_query

'\nSELECT pop.parameter_index, pop.strategy_name,\n       pop.s_t_0, pop.r1_t_0, pop.r2_t_0, pop.r12_t_0,\n       pop.s_t_45, pop.r1_t_45, pop.r2_t_45, pop.r12_t_45,\n FROM `mccoylab.dpm_dev.pop-results-2019` pop\n WHERE pop.parameter_index IN (23034022,27876071,3006527,7271268,34560328,13135956,18654825,20536176,18103583,26112099,7290258,1360409,19612425,584316,21865120,795506,3007995,3243542,23843328,33374449,38563851,15627238,32812564,37639051,19744953,16300798,33754195,37832062,37742972,24667100,18686084,21360079,25391400,37612387,31137508,15526011,29492396,18584446,9035252,28452304,37869810,22197312,30229821,14805411,32904064,12332988,34448038,9829697,28672068,33645328)\n'

In [10]:
stateVar_df = client.query(stateVar_query).to_dataframe()
stateVar_df

,parameter_index,strategy_name,s_t_0,r1_t_0,r2_t_0,r12_t_0,s_t_45,r1_t_45,r2_t_45,r12_t_45
0,9829697,0.0,1.1087e+13,1.2794e+04,2.2068e+13,1.4117e+03,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
1,9829697,2.0,1.1087e+13,2.2507e+07,1.1115e+10,4.8967e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
2,9829697,1.0,1.1087e+13,2.2507e+07,1.1115e+10,4.8967e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
3,7290258,1.0,9.9990e-01,9.9998e-01,9.9997e-01,2.6529e-06,9.9990e-01,9.9998e-01,9.9997e-01,2.6529e-06
4,7290258,2.0,9.9990e-01,9.9998e-01,9.9997e-01,2.6529e-06,9.9990e-01,9.9998e-01,9.9997e-01,2.6529e-06
...,...,...,...,...,...,...,...,...,...,...
145,23843328,1.0,1.0189e+14,1.0215e+11,2.8980e+12,2.9054e+09,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
146,23843328,2.0,1.0189e+14,1.0215e+11,2.8980e+12,2.9054e+09,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
147,21865120,1.0,9.9988e-01,2.4535e+05,9.9997e-01,1.8120e+04,9.9988e-01,9.9998e-01,9.9997e-01,2.9817e+04
148,21865120,2.0,9.9988e-01,2.4535e+05,9.9997e-01,1.8120e+04,9.9988e-01,9.9998e-01,9.9997e-01,2.9817e+04


In [11]:
# get initial param
param_query = """
SELECT *
  FROM `mccoylab.dpm_dev.param-results-2019` param
 WHERE param.parameter_index IN (""" + paramID_list + """)
"""
param_query

'\nSELECT *\n  FROM `mccoylab.dpm_dev.param-results-2019` param\n WHERE param.parameter_index IN (23034022,27876071,3006527,7271268,34560328,13135956,18654825,20536176,18103583,26112099,7290258,1360409,19612425,584316,21865120,795506,3007995,3243542,23843328,33374449,38563851,15627238,32812564,37639051,19744953,16300798,33754195,37832062,37742972,24667100,18686084,21360079,25391400,37612387,31137508,15526011,29492396,18584446,9035252,28452304,37869810,22197312,30229821,14805411,32904064,12332988,34448038,9829697,28672068,33645328)\n'

In [12]:
param_df = client.query(param_query).to_dataframe()
param_df

,parameter_index,s_init_pop,r1_init_pop,r2_init_pop,r12_init_pop,growth_rate,s_drug1_sensitivity,s_drug2_sensitivity,r1_drug1_sensitivity,r1_drug2_sensitivity,...,r1_r2_transition_rate,r1_r12_transition_rate,r2_s_transition_rate,r2_r1_transition_rate,r2_r2_transition_rate,r2_r12_transition_rate,r12_s_transition_rate,r12_r1_transition_rate,r12_r2_transition_rate,r12_r12_transition_rate
0,32812564,4.495000e+09,5.000000e+08,5.000000e+06,0.0,0.1287,0.305790,56.628000,0.305790,5.662800e-04,...,0.0,0.0,2.154000e-10,0.0,0.0,0.0,0.0,2.154000e-10,2.154000e-10,0.0
1,32904064,4.495000e+09,5.000000e+08,5.000000e+06,0.0,0.3400,0.440000,1.621200,0.440000,1.550500e-04,...,0.0,0.0,2.154000e-06,0.0,0.0,0.0,0.0,2.154000e-06,1.000000e-07,0.0
2,13135956,5.000000e+09,5.000000e+02,5.000000e+00,0.0,0.3400,0.119480,1.621200,0.119480,0.000000e+00,...,0.0,0.0,2.154000e-10,0.0,0.0,0.0,0.0,2.154000e-10,4.642000e-05,0.0
3,23843328,4.995000e+09,5.000000e+06,0.000000e+00,0.0,0.3400,0.119480,1.621200,0.119480,1.410500e-02,...,0.0,0.0,4.642000e-05,0.0,0.0,0.0,0.0,4.642000e-05,1.000000e-07,0.0
4,24667100,4.995000e+09,5.000000e+06,5.000000e+00,0.0,0.3400,0.119480,1.621200,0.119480,1.410500e-02,...,0.0,0.0,1.000000e-07,0.0,0.0,0.0,0.0,1.000000e-07,2.154000e-10,0.0
5,23034022,4.999500e+08,5.000000e+04,4.500000e+09,0.0,0.3400,0.311470,15.574000,0.311470,1.424400e-02,...,0.0,0.0,2.154000e-06,0.0,0.0,0.0,0.0,2.154000e-06,1.000000e-11,0.0
6,18103583,5.000000e+09,5.000000e+04,0.000000e+00,0.0,0.3400,0.224400,149.600000,0.224400,0.000000e+00,...,0.0,0.0,2.154000e-10,0.0,0.0,0.0,0.0,2.154000e-10,1.000000e-03,0.0
7,18584446,4.999900e+09,5.000000e+04,5.000000e+00,0.0,0.0184,2.197300,8.096000,2.197300,7.742600e-04,...,0.0,0.0,1.000000e-03,0.0,0.0,0.0,0.0,1.000000e-03,4.642000e-09,0.0
8,31137508,4.500000e+09,5.000000e+08,5.000000e+02,0.0,0.1287,0.045228,0.613680,0.045228,5.612800e-04,...,0.0,0.0,1.000000e-07,0.0,0.0,0.0,0.0,1.000000e-07,4.642000e-09,0.0
9,38563851,4.950000e+08,4.500000e+09,5.000000e+06,0.0,0.1287,0.117900,5.895000,0.117900,5.128700e-02,...,0.0,0.0,2.154000e-06,0.0,0.0,0.0,0.0,2.154000e-06,4.642000e-09,0.0


In [15]:
benefit_df.to_csv("benefit_summary.csv", header=True, index=False)
stateVar_df.to_csv("statvar.csv", header=True, index=False)
param_df.to_csv("inputParam.csv", header=True, index=False)

In [ ]:
# generate state variable features



In [ ]:
# kmeans clustering ?

In [ ]:
# sample centroids and plot trajectories

In [ ]:
# heirarchal clustering and dendrogram (color by kmeans?)

In [ ]:
# survival plot


In [ ]:
# find example patient
examplePatient_query = """
SELECT *
  FROM `mccoylab.dpm_dev.stopt-results-2019` stopt
 WHERE stopt.strategy_2_1 > strategy_2_2
 ORDER BY RAND() LIMIT 50
"""

In [ ]:
expamplePatient_df = client.query(examplePatient_query).to_dataframe()
expamplePatient_df.columns = ['parameter_index', 'CPM','DPM','DPM_trial']
expamplePatient_df

,parameter_index,CPM,DPM,DPM_trial
0,32540194,720.0,855.0,720.0
1,26220135,135.0,180.0,135.0
2,18640783,180.0,270.0,225.0
3,13586741,630.0,810.0,630.0
4,19601406,180.0,225.0,180.0
5,9610425,720.0,1845.0,1035.0
6,21984019,405.0,450.0,405.0
7,35963291,720.0,1755.0,810.0
8,1347977,180.0,270.0,225.0
9,25114108,630.0,765.0,630.0


In [ ]:
patientID = '31731026'
#patientID = '37845727'

In [ ]:
# Richa, start here
# individual plot

#patientID = '31731026'

param_query = """
SELECT *
  FROM `mccoylab.dpm_dev.param-results-2019` param
 WHERE param.parameter_index IN (""" + patientID + """)
"""

dosage_query  = """
SELECT *
  FROM `mccoylab.dpm_dev.dosage-results-2019` dosage
 WHERE dosage.parameter_index IN (""" + patientID + """)
"""

population_query  = """
SELECT *
  FROM `mccoylab.dpm_dev.pop-results-2019` pop
 WHERE pop.parameter_index IN (""" + patientID + """)
"""

param_df = client.query(param_query).to_dataframe()
dosage_df = client.query(dosage_query).to_dataframe()
population_df = client.query(population_query).to_dataframe()

# CPM: strategyID = 0
# DPM: strategyID = 1

drug1_columns = [ x for x in dosage_df.columns.tolist() if "drug1_" in x ]

drug1_df = dosage_df[['strategy_name'] + drug1_columns]
drug1_CPM = drug1_df[drug1_df['strategy_name'].astype(str) == '0'].reset_index()[drug1_columns].loc[0].tolist()
drug1_DPM = drug1_df[drug1_df['strategy_name'].astype(str) == '1'].reset_index()[drug1_columns].loc[0].tolist()

# CPM: strategyID = 0.0
# DPM: strategyID = 1.0
# DPM_trial: strategyID = 2.0

initial_pop = param_df.loc[0][['s_init_pop','r1_init_pop','r2_init_pop','r12_init_pop']].tolist()
s_pop_columns = [ x for x in population_df.columns.tolist() if "s_" in x ]
r1_pop_columns = [ x for x in population_df.columns.tolist() if "r1_" in x ]
r2_pop_columns = [ x for x in population_df.columns.tolist() if "r2_" in x ]
r12_pop_columns = [ x for x in population_df.columns.tolist() if "r12_" in x ]

s_pop_df = population_df[['strategy_name'] + s_pop_columns]
r1_pop_df = population_df[['strategy_name'] + r1_pop_columns]
r2_pop_df = population_df[['strategy_name'] + r2_pop_columns]
r12_pop_df = population_df[['strategy_name'] + r12_pop_columns]

s_pop_CPM = [initial_pop[0]]  + s_pop_df[s_pop_df['strategy_name'].astype(str) == '0.0'].reset_index()[s_pop_columns].loc[0].tolist()
r1_pop_CPM = [initial_pop[1]]  + r1_pop_df[r1_pop_df['strategy_name'].astype(str) == '0.0'].reset_index()[r1_pop_columns].loc[0].tolist()
r2_pop_CPM = [initial_pop[2]]  + r2_pop_df[r2_pop_df['strategy_name'].astype(str) == '0.0'].reset_index()[r2_pop_columns].loc[0].tolist()
r12_pop_CPM = [initial_pop[3]]  + r12_pop_df[r12_pop_df['strategy_name'].astype(str) == '0.0'].reset_index()[r12_pop_columns].loc[0].tolist()

s_pop_DPM = [initial_pop[0]]  + s_pop_df[s_pop_df['strategy_name'].astype(str) == '1.0'].reset_index()[s_pop_columns].loc[0].tolist()
r1_pop_DPM = [initial_pop[1]]  + r1_pop_df[r1_pop_df['strategy_name'].astype(str) == '1.0'].reset_index()[r1_pop_columns].loc[0].tolist()
r2_pop_DPM = [initial_pop[2]]  + r2_pop_df[r2_pop_df['strategy_name'].astype(str) == '1.0'].reset_index()[r2_pop_columns].loc[0].tolist()
r12_pop_DPM = [initial_pop[3]]  + r12_pop_df[r12_pop_df['strategy_name'].astype(str) == '1.0'].reset_index()[r12_pop_columns].loc[0].tolist()

s_pop_DPMtrial = [initial_pop[0]]  + s_pop_df[s_pop_df['strategy_name'].astype(str) == '2.0'].reset_index()[s_pop_columns].loc[0].tolist()
r1_pop_DPMtrial = [initial_pop[1]]  + r1_pop_df[r1_pop_df['strategy_name'].astype(str) == '2.0'].reset_index()[r1_pop_columns].loc[0].tolist()
r2_pop_DPMtrial = [initial_pop[2]]  + r2_pop_df[r2_pop_df['strategy_name'].astype(str) == '2.0'].reset_index()[r2_pop_columns].loc[0].tolist()
r12_pop_DPMtrial = [initial_pop[3]]  + r12_pop_df[r12_pop_df['strategy_name'].astype(str) == '2.0'].reset_index()[r12_pop_columns].loc[0].tolist()

timestep = [ x*45 for x in range(0,len(s_pop_CPM))]
patientID_list = [patientID] * len(s_pop_CPM)
individualResults_df = pd.DataFrame(list(zip(patientID_list, timestep, s_pop_CPM,r1_pop_CPM,r2_pop_CPM,r12_pop_CPM,
                                    s_pop_DPM,r1_pop_DPM,r2_pop_DPM,r12_pop_DPM,
                                    s_pop_DPMtrial,r1_pop_DPMtrial,r2_pop_DPMtrial,r12_pop_DPMtrial,
                                    drug1_CPM, drug1_DPM,
                                )),columns=['patientID','timestep', 's_pop_CPM', 'r1_pop_CPM', 'r2_pop_CPM', 'r12_pop_CPM',
                                            's_pop_DPM','r1_pop_DPM','r2_pop_DPM','r12_pop_DPM',
                                            's_pop_DPMtrial','r1_pop_DPMtrial','r2_pop_DPMtrial','r12_pop_DPMtrial',
                                            'drug1_CPM', 'drug1_DPM',
                                            ])

individualResults_df

,patientID,timestep,s_pop_CPM,r1_pop_CPM,r2_pop_CPM,r12_pop_CPM,s_pop_DPM,r1_pop_DPM,r2_pop_DPM,r12_pop_DPM,s_pop_DPMtrial,r1_pop_DPMtrial,r2_pop_DPMtrial,r12_pop_DPMtrial,drug1_CPM,drug1_DPM
0,31731026,0,4500000000.0,500000000.0,50000.0,0.0,4500000000.0,500000000.0,50000.0,0.0,4500000000.0,500000000.0,50000.0,0.0,1.00,1.00
1,31731026,45,1.9867e+08,2.2240e+07,2.1276e+05,1.1101e+04,1.9867e+08,2.2240e+07,2.1276e+05,1.1101e+04,1.9867e+08,2.2240e+07,2.1276e+05,1.1101e+04,1.00,1.00
2,31731026,90,8.7716e+06,9.8916e+05,4.9127e+05,2.6300e+04,8.7716e+06,9.8916e+05,4.9127e+05,2.6300e+04,8.7716e+06,9.8916e+05,4.9127e+05,2.6300e+04,1.00,0.50
3,31731026,135,3.8727e+05,4.3992e+04,1.1245e+06,6.1144e+04,3.8727e+05,2.6745e+05,1.5621e+05,5.1423e+04,3.8727e+05,2.6745e+05,1.5621e+05,5.1423e+04,1.00,0.50
4,31731026,180,1.7098e+04,1.9564e+03,2.5737e+06,1.4207e+05,1.7098e+04,7.2109e+04,4.9657e+04,9.9891e+04,1.7098e+04,7.2109e+04,4.9657e+04,9.9891e+04,1.00,0.50
5,31731026,225,7.5489e+02,8.7003e+01,5.8902e+06,3.3003e+05,7.5489e+02,1.9433e+04,1.5785e+04,1.9384e+05,7.5489e+02,1.9433e+04,1.5785e+04,1.9384e+05,0.00,0.00
6,31731026,270,3.3329e+01,1.4337e+02,2.6006e+05,5.4494e+05,3.3329e+01,3.1961e+04,6.9691e+02,3.1881e+05,3.3329e+01,5.2366e+03,5.0176e+03,3.7608e+05,0.00,0.00
7,31731026,315,1.4715e+00,2.3580e+02,1.1482e+04,8.9634e+05,1.4715e+00,5.2565e+04,3.0769e+01,5.2434e+05,1.4715e+00,1.4111e+03,1.5950e+03,7.2965e+05,0.00,0.00
8,31731026,360,9.9996e-01,3.8782e+02,5.0692e+02,1.4742e+06,9.9996e-01,8.6452e+04,1.3585e+00,8.6236e+05,9.9996e-01,2.3208e+03,7.0419e+01,1.2000e+06,0.00,0.00
9,31731026,405,9.9996e-01,6.3784e+02,2.2381e+01,2.4246e+06,9.9996e-01,1.4219e+05,9.9999e-01,1.4183e+06,9.9996e-01,3.8169e+03,3.1090e+00,1.9737e+06,1.00,0.00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
directory = '/content/drive/MyDrive/dpmTrialResults/Evolutionary Classifier/'
#filename = directory + "individualResults_" + patientID + ".csv"
filename = "individualResults_" + patientID + ".csv"

In [ ]:
individualResults_df.to_csv(filename, sep = ',', index=False)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib as mpl


def DPM_miscellaneous_treatment_change_time(d):
    d_diff = np.diff(d)
    d_diff_boolen = d_diff.astype(bool)
    if d_diff_boolen.ndim != 1:
        pos_changedose = np.sum(d_diff_boolen, axis=0).astype(bool)
    else:
        pos_changedose = d_diff_boolen
    pos_changedose, = np.where(pos_changedose)
    # Position where drug dose changes.
    pos_changedose += 1
    # At the beginning, index=0, drug dose always changes. It can be seen as the start of applying treatment, previous drug doses are all 0.
    pos_changedose = np.insert(pos_changedose, 0, 0)
    return pos_changedose


def DPM_miscellaneous_cutbydose(drug, Spop, R1pop, R2pop, R12pop):
    index, = np.where(drug < 0)
    if index.any():
        index = index[0]
        drug = drug[:index]
        Spop = Spop[:index+1]
        R1pop = R1pop[:index+1]
        R2pop = R2pop[:index+1]
        R12pop = R12pop[:index+1]
    return drug, Spop, R1pop, R2pop, R12pop


def DPM_miscellaneous_cutbycell(drug, Spop, R1pop, R2pop, R12pop, Limit_mortality):
    X = np.vstack((Spop, R1pop, R2pop, R12pop))
    total = X.sum(axis=0)
    maxcell = np.amax(X, axis=0)
    index_total, = np.where(total > Limit_mortality)
    index_maxcell, = np.where(maxcell < 1)
    index_total = index_total[0] if index_total.any() else index_total
    index_maxcell = index_maxcell[0] if index_maxcell.any() else index_maxcell
    index = []
    if index_total.any() and index_maxcell.any():
        index = min([index_total, index_maxcell])
    elif index_total.any():
        index = index_total
    elif index_maxcell.any():
        index = index_maxcell
    if index.any():
        drug = drug[:index-1]
        Spop = Spop[:index]
        R1pop = R1pop[:index]
        R2pop = R2pop[:index]
        R12pop = R12pop[:index]
    return drug, Spop, R1pop, R2pop, R12pop


def DPM_miscellaneous_dataplot(drug1, drug2, Spop, R1pop, R2pop, R12pop, timestep):
    t = np.arange(0, Spop.shape[0]*timestep, timestep)
    X = np.vstack((Spop, R1pop, R2pop, R12pop))
    d = np.vstack((drug1, drug2))
    data = (t, X, d)
    return data


def DPM_plot_1strategy(data, title='', savename='ex', Num_drug=2, Simduration=1800, Limit_moleculardetection=1/1e4,
                       Limit_radiologicdetection=1e9, Limit_mortality=1e13):
    mpl.use('pdf')
    # Plot simulation result of one strategy.
    LOGBASE = 10
    MINX = .1
    XMINVAL = -1
    XINCREASE = 45
    YINCREASE = 2
    YMINVAL = 1e-2
    TEXTFONT = 10
    LEGEND_BOXANCHOR = (0.5, 1.15)
    color_X = ('b', 'g', 'c', 'm', 'r')
    label_X = ('Total cells', 'S cells', 'R1 cells', 'R2 cells', 'R12 cells')
    color_drug = ('g', 'b')
    label_drug = ('Drug 1', 'Drug 2')
    legend_colnum = 4
    legend_order = np.reshape(np.array(range(0, 8)), (-1, legend_colnum)).flatten('F')
    legend_order = legend_order[:-1]

    t, X, d = data
    Xtotal = 1.5 * np.sum(X, axis=0)
    X = np.vstack((Xtotal, X))
    diffpts = DPM_miscellaneous_treatment_change_time(d)

    plt.rcParams['font.size'] = 14
    # plt.figure()
    # ax = plt.axes()
    fig, ax = plt.subplots()
    # fig.set_size_inches(fig_width-1, fig_height+1)
    fig.set_size_inches(14, 9)

    ax.set_yscale('log', base=LOGBASE)
    for i_type in range(X.shape[0]):
        X1 = X[i_type, :]
        X1[X1 < MINX] = MINX
        plt.plot(t, X1, color=color_X[i_type], label=label_X[i_type])

    ymaxval = math.ceil(math.log(np.max(X), LOGBASE)) + YINCREASE
    # xmaxval = t[-1]/T_NORM

    X_Limit_moleculardetection = X[0, ] * Limit_moleculardetection
    plt.plot(t, X_Limit_moleculardetection, color='k', linestyle='dotted')
    if ymaxval > math.log(Limit_radiologicdetection, LOGBASE):
        plt.axhline(y=Limit_radiologicdetection, color='k', linestyle='dashdot')
        plt.text(np.max(t)/2, 2 * Limit_radiologicdetection, 'Limit of radiologic detection', fontsize=TEXTFONT)
    if ymaxval > math.log(Limit_mortality, LOGBASE):
        plt.axhline(y=Limit_mortality, color='k', linestyle='dashed')
        plt.text(np.max(t)/2, 2 * Limit_mortality, 'Limit of mortality', fontsize=TEXTFONT)

    diffpts = np.append(diffpts, (len(t)-1))
    diffpts = diffpts.astype(int)
    for i_treat in range(len(diffpts)-1):
        d_i = d[:, diffpts[i_treat]:diffpts[i_treat+1]]
        d_i = np.unique(d_i, axis=1)
        d_i_sum = d_i.sum()
        y_basal = ymaxval - YINCREASE
        for i_drug in range(Num_drug):
            if d_i.shape[1] != 0 and d_i[i_drug] != 0:
                i_begin = diffpts[i_treat]  # if i_treat == 0 else diffpts[i_treat]-1
                plt.fill_between(t[i_begin:diffpts[i_treat + 1]+1], LOGBASE ** y_basal,
                                 LOGBASE ** (y_basal + YINCREASE * d_i[i_drug]/d_i_sum), color=color_drug[i_drug])
                plt.hlines(y=LOGBASE ** (y_basal + YINCREASE * d_i[i_drug] / d_i_sum), xmin=t[i_begin],
                           xmax=t[diffpts[i_treat + 1]], color='black')
                if i_treat != 0:
                    plt.vlines(x=t[i_begin], ymin=LOGBASE ** (ymaxval-YINCREASE), ymax=LOGBASE ** ymaxval, color='black')
                y_basal = y_basal + YINCREASE * d_i[i_drug]/d_i_sum

    # Plot artificial label.
    y_basal = ymaxval - YINCREASE
    d_lab = np.full(Num_drug, 1/Num_drug, dtype=float)
    t_lab = np.array(range(-int(t[-1]), -int(t[-1]/2), 1))
    for i in range(Num_drug):
        plt.fill_between(t_lab, LOGBASE ** y_basal, LOGBASE ** (y_basal + YINCREASE * d_lab[i]), color=color_drug[i], label=label_drug[i])
        y_basal = y_basal + YINCREASE * d_lab[i]
    plt.plot(t_lab, t_lab * Limit_moleculardetection, color='k', linestyle='dotted')
    plt.xlabel('Time[Months]')
    plt.ylabel('Number of cells')
    plt.title(title)
    handles, labels = plt.gca().get_legend_handles_labels()
    leg = plt.legend([handles[idx] for idx in legend_order], [labels[idx] for idx in legend_order],
                     bbox_to_anchor=LEGEND_BOXANCHOR, loc='upper center', ncol=legend_colnum)
    leg.get_frame().set_edgecolor('k')
    leg.get_frame().set_linewidth(0)
    # plt.xlim([XMINVAL, xmaxval + XINCREASE])
    plt.xlim([XMINVAL, Simduration + XINCREASE])
    plt.ylim([YMINVAL, LOGBASE ** ymaxval])
    ylocmaj = mpl.ticker.LogLocator(base=LOGBASE, numticks=len(range(-2, int(ymaxval), 2)))
    ax.yaxis.set_major_locator(ylocmaj)
    ylocmin = mpl.ticker.LogLocator(base=LOGBASE, subs=np.arange(0, LOGBASE) * 1/LOGBASE, numticks=100)
    ax.yaxis.set_minor_locator(ylocmin)
    ax.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())
    # ax.set_xticks(t)
    plt.show()
    plt.savefig(directory + savename+'.pdf', dpi=300)
    return


def DPM_plot_1csv(filename='./singleResults.csv'):
    savename = 'ex'
    df = pd.read_csv(filename)
    colnum_drug1 = -1
    colnum_Spop = 1
    colnum_R1pop = 2
    colnum_R2pop = 3
    colnum_R12pop = 4
    timestep = 45
    drug1 = df.iloc[:, colnum_drug1].to_numpy()
    Spop = df.iloc[:, colnum_Spop].to_numpy()
    R1pop = df.iloc[:, colnum_R1pop].to_numpy()
    R2pop = df.iloc[:, colnum_R2pop].to_numpy()
    R12pop = df.iloc[:, colnum_R12pop].to_numpy()

    drug1, Spop, R1pop, R2pop, R12pop = DPM_miscellaneous_cutbydose(drug1, Spop, R1pop, R2pop, R12pop)

    drug2 = 1 - drug1
    t = np.arange(0, Spop.shape[0]*timestep, timestep)
    X = np.vstack((Spop, R1pop, R2pop, R12pop))
    d = np.vstack((drug1, drug2))
    data = (t, X, d)
    title = ''
    DPM_plot_1strategy(data)


def DPM_plot_csv(filename='./individualResults_31731026.csv'):
    savename = 'ex'
    df = pd.read_csv(filename)
    timestep = 45
    colnum_drug1_cpm = 13
    colnum_drug1_dpm = 14
    colnum_Spop_cpm, colnum_R1pop_cpm, colnum_R2pop_cpm, colnum_R12pop_cpm = 1, 2, 3, 4
    colnum_Spop_dpm, colnum_R1pop_dpm, colnum_R2pop_dpm, colnum_R12pop_dpm = 5, 6, 7, 8
    colnum_Spop_dpmtrial, colnum_R1pop_dpmtrial, colnum_R2pop_dpmtrial, colnum_R12pop_dpmtrial = 9, 10, 11, 12

    drug1_cpm = df.iloc[:, colnum_drug1_cpm].to_numpy()
    drug1_dpm = df.iloc[:, colnum_drug1_dpm].to_numpy()

    Spop_cpm = df.iloc[:, colnum_Spop_cpm].to_numpy()
    R1pop_cpm = df.iloc[:, colnum_R1pop_cpm].to_numpy()
    R2pop_cpm = df.iloc[:, colnum_R2pop_cpm].to_numpy()
    R12pop_cpm = df.iloc[:, colnum_R12pop_cpm].to_numpy()

    Spop_dpm = df.iloc[:, colnum_Spop_dpm].to_numpy()
    R1pop_dpm = df.iloc[:, colnum_R1pop_dpm].to_numpy()
    R2pop_dpm = df.iloc[:, colnum_R2pop_dpm].to_numpy()
    R12pop_dpm = df.iloc[:, colnum_R12pop_dpm].to_numpy()

    Spop_dpmtrial, R1pop_dpmtrial, R2pop_dpmtrial, R12pop_dpmtrial = \
        (df.iloc[:, colnum_Spop_dpmtrial].to_numpy(),
         df.iloc[:, colnum_R1pop_dpmtrial].to_numpy(),
         df.iloc[:, colnum_R2pop_dpmtrial].to_numpy(),
         df.iloc[:, colnum_R12pop_dpmtrial].to_numpy())

    drug1_dpmtrial = np.concatenate((drug1_dpm[:2], drug1_cpm[2:]))
    drug1_cpm, Spop_cpm, R1pop_cpm, R2pop_cpm, R12pop_cpm = \
        DPM_miscellaneous_cutbydose(drug1_cpm, Spop_cpm, R1pop_cpm, R2pop_cpm, R12pop_cpm)
    drug1_dpm, Spop_dpm, R1pop_dpm, R2pop_dpm, R12pop_dpm = \
        DPM_miscellaneous_cutbydose(drug1_dpm, Spop_dpm, R1pop_dpm, R2pop_dpm, R12pop_dpm)

    Limit_mortality = 1e13
    drug1_dpmtrial, Spop_dpmtrial, R1pop_dpmtrial, R2pop_dpmtrial, R12pop_dpmtrial = \
        DPM_miscellaneous_cutbycell(drug1_dpmtrial, Spop_dpmtrial, R1pop_dpmtrial, R2pop_dpmtrial, R12pop_dpmtrial, Limit_mortality)

    drug2_cpm, drug2_dpm, drug2_dpmtrial = 1 - drug1_cpm, 1 - drug1_dpm, 1- drug1_dpmtrial
    data_cpm = DPM_miscellaneous_dataplot(drug1_cpm, drug2_cpm, Spop_cpm, R1pop_cpm, R2pop_cpm, R12pop_cpm, timestep)
    data_dpm = DPM_miscellaneous_dataplot(drug1_dpm, drug2_dpm, Spop_dpm, R1pop_dpm, R2pop_dpm, R12pop_dpm, timestep)
    data_dpmtrial = DPM_miscellaneous_dataplot(drug1_dpmtrial, drug2_dpmtrial, Spop_dpmtrial, R1pop_dpmtrial, R2pop_dpmtrial, R12pop_dpmtrial,
                                               timestep)

    DPM_plot_1strategy(data_cpm, title='cpm', savename='cpm')
    DPM_plot_1strategy(data_dpm, title='dpm', savename='dpm')
    DPM_plot_1strategy(data_dpmtrial, title='dpmtrail', savename='dpmtrail')
    return



In [ ]:
filename

'/content/drive/MyDrive/dpmTrialResults/Evolutionary Classifier/individualResults_31731026.csv'

In [ ]:
DPM_plot_csv(filename=filename)
#'./individualResults_31731026.csv')
#DPM_plot_1csv(filename='./singleResults.csv')


AttributeError: ignored

In [ ]:
plot = sns.lineplot(data = pop_plot_df_melt, x = 'timestep',y = 'size', hue='population')

plot.set_yscale('log')

NameError: ignored